# Before Building Model

In [ ]:
from IPython.display import clear_output
!pip install pycaret --user
clear_output()

import numpy as np
import pandas as pd
import os,random,warnings
warnings.simplefilter('ignore')
from sklearn import preprocessing

import pycaret
from pycaret.classification import *

TRAIN_PATH = "../input/spaceship-titanic/train.csv"
TEST_PATH = "../input/spaceship-titanic/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/spaceship-titanic/sample_submission.csv"
SUBMISSION_PATH = "submission.csv"

ID = "PassengerId"
TARGET = "Transported"

SEED = 2022
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything()

PYCARET_TARGET_NAME = "Label"

train = pd.read_csv(TRAIN_PATH)
train[TARGET] = train[TARGET].astype(int)
test = pd.read_csv(TEST_PATH)

# Build Model

In [ ]:
# setup?

In [ ]:
setup(
    data = train, 
    target = TARGET,
    silent = True, 
    normalize = True, 
    session_id=1, 
    data_split_stratify=True,
    use_gpu=True,
    numeric_imputation="median",
    remove_multicollinearity=True,
    fix_imbalance=True
)

In [ ]:
FOLD = 3
BLEND_FOLD=3
TUNE_FOLD=3
CRITERIA="Accuracy"

In [ ]:
# top3 = compare_models(n_select=3, include=["xgboost","catboost","lightgbm","ada","ridge","rf"],fold = FOLD)
# top3

In [ ]:
BLEND_MODELS=["catboost","xgboost","lightgbm"]

In [ ]:
train_models = []
for model in BLEND_MODELS:
    trainedModel = create_model(model,fold = FOLD)
    train_models.append(trainedModel)

In [ ]:
blendModel = blend_models(estimator_list = train_models,optimize=CRITERIA,fold = BLEND_FOLD)
predict_model(blendModel)

In [ ]:
tuneModel = tune_model(blendModel,fold=TUNE_FOLD)
predict_model(tuneModel)

In [ ]:
final_model = finalize_model(tuneModel)
predict_model(final_model)

In [ ]:
plot_model(final_model, "confusion_matrix")

In [ ]:
plot_model(final_model, "error")

In [ ]:
plot_model(final_model, "boundary")

# After Building Model

In [ ]:
predictions = predict_model(final_model, data=test)

predictions[PYCARET_TARGET_NAME] = predictions[PYCARET_TARGET_NAME].astype(bool)
submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission[TARGET] = predictions[PYCARET_TARGET_NAME]
submission.to_csv(SUBMISSION_PATH, index=False)
submission.head()